In [1]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import utils,layers,Sequential
import cv2
import tensorflow_hub as hub

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train = utils.image_dataset_from_directory('/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres', image_size = (256,256))

def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train = train.map(process)

Found 1856 files belonging to 2 classes.


In [3]:
model = Sequential([
    layers.Input(shape=(256,256,3)),
    hub.KerasLayer('https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/5'),
    layers.Dense(128,activation="relu"),
    layers.Dense(80,activation='softmax')
])

In [4]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.02),
    metrics=['Accuracy']
)

earlystopping=tf.keras.callbacks.EarlyStopping(
    monitor='Accuracy',
    min_delta=0.0001,
    patience=2,
    restore_best_weights=True,
    start_from_epoch=15
)

tensorboard= tf.keras.callbacks.TensorBoard()

earlyStop = tf.keras.callbacks.ModelCheckpoint(
    'eStop.ckpt',
    monitor = 'Accuracy',
    save_best_only = True,
    mode = 'auto',
    save_freq='epoch',
)

In [5]:
 with tf.device('/GPU:0'):
    model.fit(train,
              epochs=50,
              callbacks=[earlystopping,tensorboard,earlyStop])

Epoch 1/50
58/58 [==============================] - 126s 2s/step - loss: 1.3441 - Accuracy: 0.8303
Epoch 2/50
58/58 [==============================] - 59s 885ms/step - loss: 0.2443 - Accuracy: 0.9106
Epoch 3/50
58/58 [==============================] - 61s 918ms/step - loss: 0.2018 - Accuracy: 0.9181
Epoch 4/50
58/58 [==============================] - 60s 899ms/step - loss: 0.2211 - Accuracy: 0.9375
Epoch 5/50
58/58 [==============================] - 59s 883ms/step - loss: 0.1593 - Accuracy: 0.9461
Epoch 6/50
58/58 [==============================] - 61s 897ms/step - loss: 0.1310 - Accuracy: 0.9520
Epoch 7/50
58/58 [==============================] - 60s 903ms/step - loss: 0.1098 - Accuracy: 0.9569
Epoch 8/50
58/58 [==============================] - 60s 910ms/step - loss: 0.1098 - Accuracy: 0.9564
Epoch 9/50
58/58 [==============================] - 60s 912ms/step - loss: 0.0679 - Accuracy: 0.9752
Epoch 10/50
58/58 [==============================] - 62s 921ms/step - loss: 0.1023 - Accuracy

In [6]:
model.save('/kaggle/working/tyreDetection.keras')